In [11]:
! pip install backtesting

In [12]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ParameterGrid
from backtesting import Backtest, Strategy
from backtesting.test import SMA

In [13]:
# Pobieranie danych historycznych dla MSFT
ticker = 'MSFT'
data = yf.download(ticker, start='2020-01-01', end='2024-01-01')
data.reset_index(inplace=True)
data = data[['Date', 'Close']]
data.columns = ['ds', 'y']

[*********************100%%**********************]  1 of 1 completed


In [14]:
# Parametry do optymalizacji
param_grid = {
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

In [21]:
# Wybór najlepszych parametrów do modelu
best_params = None
best_mae = float('inf')

for params in ParameterGrid(param_grid):
    model = Prophet(**params, daily_seasonality=True)
    model.fit(data)

    future = model.make_future_dataframe(periods=126)  # Zakładając prognozowanie na 2024-01-01 do 2024-05-06
    forecast = model.predict(future)

    forecast_actual = forecast[['ds', 'yhat']].merge(data, on='ds', how='inner')
    mae = mean_absolute_error(forecast_actual['y'], forecast_actual['yhat'])

    if mae < best_mae:
        best_mae = mae
        best_params = params

print(f"Best parameters: {best_params}")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/hpsr6rdn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/lihcg9ic.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=52425', 'data', 'file=/tmp/tmpj_43af70/hpsr6rdn.json', 'init=/tmp/tmpj_43af70/lihcg9ic.json', 'output', 'file=/tmp/tmpj_43af70/prophet_model0xb8hh0l/prophet_model-20240604144220.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:42:20 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:42:20 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/njwafl0r.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/do8g6bsj.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

Best parameters: {'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 10.0}


In [22]:
# Stworzenie modelu z najlepszymi parametrami
model = Prophet(**best_params)
model.fit(data)
future = pd.date_range(start='2024-01-02', end='2024-05-02')
future_df = pd.DataFrame(future, columns=['ds'])
forecast = model.predict(future_df)
forecast

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/47dylqo6.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpj_43af70/ofvfo2p5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48387', 'data', 'file=/tmp/tmpj_43af70/47dylqo6.json', 'init=/tmp/tmpj_43af70/ofvfo2p5.json', 'output', 'file=/tmp/tmpj_43af70/prophet_modelcw8gcp0y/prophet_model-20240604144249.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:42:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:42:49 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-01-02,388.817614,371.943215,391.483481,388.817614,388.817614,-7.283006,-7.283006,-7.283006,3.955830,3.955830,3.955830,-11.238836,-11.238836,-11.238836,0.0,0.0,0.0,381.534609
1,2024-01-03,389.256765,372.453226,391.474849,389.256765,389.256765,-7.448603,-7.448603,-7.448603,4.373075,4.373075,4.373075,-11.821678,-11.821678,-11.821678,0.0,0.0,0.0,381.808163
2,2024-01-04,389.695916,372.217447,391.937881,389.695916,389.695916,-8.130936,-8.130936,-8.130936,4.317273,4.317273,4.317273,-12.448209,-12.448209,-12.448209,0.0,0.0,0.0,381.564980
3,2024-01-05,390.135067,371.835158,391.084714,390.135067,390.135067,-8.718121,-8.718121,-8.718121,4.388301,4.388301,4.388301,-13.106422,-13.106422,-13.106422,0.0,0.0,0.0,381.416946
4,2024-01-06,390.574218,356.900092,376.135540,390.574218,390.574218,-24.353588,-24.353588,-24.353588,-10.570803,-10.570803,-10.570803,-13.782785,-13.782785,-13.782785,0.0,0.0,0.0,366.220630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2024-04-28,440.198275,384.263764,513.128302,375.922150,504.276796,9.135640,9.135640,9.135640,-10.570807,-10.570807,-10.570807,19.706447,19.706447,19.706447,0.0,0.0,0.0,449.333915
118,2024-04-29,440.637426,398.999161,527.254687,375.460700,505.312615,23.776304,23.776304,23.776304,4.107131,4.107131,4.107131,19.669173,19.669173,19.669173,0.0,0.0,0.0,464.413731
119,2024-04-30,441.076577,395.421111,530.053542,374.656248,507.173512,23.591767,23.591767,23.591767,3.955830,3.955830,3.955830,19.635937,19.635937,19.635937,0.0,0.0,0.0,464.668345
120,2024-05-01,441.515728,395.922985,529.844856,373.263361,509.034409,23.984114,23.984114,23.984114,4.373075,4.373075,4.373075,19.611039,19.611039,19.611039,0.0,0.0,0.0,465.499842


In [23]:
# Generowanie sygnałów handlowych
def generate_signals(forecast, threshold=0.1):
    forecast['signal'] = 0
    forecast['signal'][forecast['yhat'].diff() > threshold] = 1
    forecast['signal'][forecast['yhat'].diff() < -threshold] = -1
    return forecast

signals = generate_signals(forecast)
print(signals)

            ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0   2024-01-02  388.817614  371.943215  391.483481   388.817614   388.817614   
1   2024-01-03  389.256765  372.453226  391.474849   389.256765   389.256765   
2   2024-01-04  389.695916  372.217447  391.937881   389.695916   389.695916   
3   2024-01-05  390.135067  371.835158  391.084714   390.135067   390.135067   
4   2024-01-06  390.574218  356.900092  376.135540   390.574218   390.574218   
..         ...         ...         ...         ...          ...          ...   
117 2024-04-28  440.198275  384.263764  513.128302   375.922150   504.276796   
118 2024-04-29  440.637426  398.999161  527.254687   375.460700   505.312615   
119 2024-04-30  441.076577  395.421111  530.053542   374.656248   507.173512   
120 2024-05-01  441.515728  395.922985  529.844856   373.263361   509.034409   
121 2024-05-02  441.954879  394.880801  535.546476   372.453972   510.446122   

     additive_terms  additive_terms_low

<ipython-input-23-cc11e9b9171a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['signal'][forecast['yhat'].diff() > threshold] = 1
<ipython-input-23-cc11e9b9171a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast['signal'][forecast['yhat'].diff() < -threshold] = -1


In [24]:
# Backtesting
class ProphetStrategy(Strategy):
    def init(self):
        self.forecast = signals.set_index('ds')

    def next(self):
        date = self.data.index[-1]

        if date in self.forecast.index:
            signal = self.forecast.loc[date]['signal']

            if signal == 1:
                self.buy(sl=self.data.Close[-1] * 0.95, tp=self.data.Close[-1] * 1.06)  # Stop-loss 5%, Take-profit 6%
            elif signal == -1:
                self.sell(sl=self.data.Close[-1] * 1.05, tp=self.data.Close[-1] * 0.95)  # Stop-loss 5%, Take-profit 5%

In [25]:
# Przygotowanie danych do backtestu
bt_data = yf.download(ticker, start='2024-01-02', end='2024-05-02')

bt = Backtest(bt_data, ProphetStrategy, cash=10000)
stats = bt.run()
bt.plot()

[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1969', ...)

In [26]:
print(stats)

Start                     2024-01-02 00:00:00
End                       2024-05-01 00:00:00
Duration                    120 days 00:00:00
Exposure Time [%]                   97.619048
Equity Final [$]                 10207.048867
Equity Peak [$]                  11896.876634
Return [%]                           2.070489
Buy & Hold Return [%]                6.490147
Return (Ann.) [%]                    6.340961
Volatility (Ann.) [%]               23.376011
Sharpe Ratio                         0.271259
Sortino Ratio                        0.383783
Calmar Ratio                          0.41221
Max. Drawdown [%]                  -15.382846
Avg. Drawdown [%]                   -3.232724
Max. Drawdown Duration       41 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       6.625645
Worst Trade [%]                     -4.745621
Avg. Trade [%]                    